In [42]:
import ccxt 
import os
from dotenv import load_dotenv
import pprint
import pandas as pd
import numpy as np

In [3]:
load_dotenv()
def load_bingx():
    
    api_key = os.getenv("api_key")
    api_secret = os.getenv("api_secret")
    bingx = ccxt.bingx({
    'apiKey': api_key,
    'secret': api_secret,
    'options': {
        'defaultType': 'spot'}  # MUY IMPORTANTE para usar futuros
    
    })
    return bingx
bingx = load_bingx()
print(bingx.fetch_balance())

{'info': {'code': '0', 'msg': '', 'debugMsg': '', 'retryable': '0', 'data': {'balances': [{'asset': 'ETH', 'disPlayName': 'ETH', 'free': '0.000073575', 'locked': '0'}, {'asset': 'BTC', 'disPlayName': 'BTC', 'free': '1.23190299e-8', 'locked': '0'}, {'asset': 'USDT', 'disPlayName': 'USDT', 'free': '1.5376685963944254', 'locked': '14.25'}, {'asset': 'VST', 'disPlayName': 'VST', 'free': '166240.09555901', 'locked': '0'}, {'asset': 'ARB', 'disPlayName': 'ARB', 'free': '0', 'locked': '0'}, {'asset': 'NEO', 'disPlayName': 'NEO', 'free': '0', 'locked': '0'}, {'asset': 'USDC', 'disPlayName': 'USDC', 'free': '0', 'locked': '0'}, {'asset': 'BNB', 'disPlayName': 'BNB', 'free': '0.0005', 'locked': '0'}, {'asset': 'DOT', 'disPlayName': 'DOT', 'free': '0', 'locked': '0'}, {'asset': 'LTC', 'disPlayName': 'LTC', 'free': '0', 'locked': '0'}]}}, 'ETH': {'free': 7.3575e-05, 'used': 0.0, 'total': 7.3575e-05}, 'BTC': {'free': 1.23190299e-08, 'used': 0.0, 'total': 1.23190299e-08}, 'USDT': {'free': 1.53766859

In [50]:
pprint.pprint(bingx.fetch_ticker("USDC/USDT"))

{'ask': 0.9994,
 'askVolume': 38245914.652,
 'average': 0.9993,
 'baseVolume': 43286016.37,
 'bid': 0.9993,
 'bidVolume': 15003.434,
 'change': 0.0001,
 'close': 0.9994,
 'datetime': '2025-07-18T23:15:08.633Z',
 'high': 0.9996,
 'indexPrice': None,
 'info': {'askPrice': '0.9994',
          'askQty': '38245914.652',
          'bidPrice': '0.9993',
          'bidQty': '15003.434',
          'closeTime': '1752880508633',
          'highPrice': '0.9996',
          'lastPrice': '0.9994',
          'lowPrice': '0.9990',
          'openPrice': '0.9993',
          'openTime': '1752794108633',
          'priceChange': '0.0001',
          'priceChangePercent': '0.01%',
          'quoteVolume': '43254968.79',
          'symbol': 'USDC-USDT',
          'volume': '43286016.37'},
 'last': 0.9994,
 'low': 0.999,
 'markPrice': None,
 'open': 0.9993,
 'percentage': 0.01,
 'previousClose': None,
 'quoteVolume': 43254968.79,
 'symbol': 'USDC/USDT',
 'timestamp': 1752880508633,
 'vwap': 0.9992827341806044

In [76]:
data = bingx.fetch_ohlcv("USDC/USDT","5m")
        #print(simple_arbitrage_usdc_usdt(data))
data= pd.DataFrame(data)
data.rename(columns={0: "timestamp", 1: "open", 2: "high", 3: "low", 4: "close", 5: "volume"}, inplace=True)
data['datetime'] = pd.to_datetime(data['timestamp'], unit='ms')

In [86]:
data

,timestamp,open,high,low,close,volume,datetime
0,1752738000000,0.9995,0.9996,0.9995,0.9995,145930.219,2025-07-17 07:40:00
1,1752738300000,0.9995,0.9996,0.9994,0.9994,128544.469,2025-07-17 07:45:00
2,1752738600000,0.9994,0.9995,0.9994,0.9995,37689.820,2025-07-17 07:50:00
3,1752738900000,0.9995,0.9996,0.9994,0.9994,39160.969,2025-07-17 07:55:00
4,1752739200000,0.9994,0.9995,0.9994,0.9994,145669.758,2025-07-17 08:00:00
...,...,...,...,...,...,...,...
495,1752886500000,0.9993,0.9994,0.9993,0.9993,36630.544,2025-07-19 00:55:00
496,1752886800000,0.9993,0.9994,0.9993,0.9993,52430.297,2025-07-19 01:00:00
497,1752887100000,0.9993,0.9993,0.9993,0.9993,46232.538,2025-07-19 01:05:00
498,1752887400000,0.9993,0.9994,0.9993,0.9993,32610.436,2025-07-19 01:10:00


In [49]:
bingx.create_order(
    symbol='USDC/USDT',
    type='limit',
    side='buy',
    amount=15,
    price=0.95)

{'info': {'symbol': 'USDC-USDT',
  'orderId': '1946347186262278144',
  'transactTime': '1752880305243',
  'price': '0.95',
  'stopPrice': '0',
  'origQty': '15',
  'executedQty': '0',
  'cummulativeQuoteQty': '0',
  'status': 'PENDING',
  'type': 'LIMIT',
  'side': 'BUY',
  'clientOrderID': '',
  'clientUserID': '',
  'msg': '',
  'bxUid': ''},
 'id': '1946347186262278144',
 'clientOrderId': None,
 'symbol': 'USDC/USDT',
 'timestamp': 1752880305243,
 'datetime': '2025-07-18T23:11:45.243Z',
 'lastTradeTimestamp': None,
 'lastUpdateTimestamp': None,
 'type': 'limit',
 'timeInForce': None,
 'postOnly': None,
 'side': 'buy',
 'price': 0.95,
 'triggerPrice': None,
 'stopLossPrice': None,
 'takeProfitPrice': None,
 'average': None,
 'cost': 0.0,
 'amount': 15.0,
 'filled': 0.0,
 'remaining': 15.0,
 'status': 'open',
 'fee': {'currency': 'USDC', 'cost': None},
 'trades': [],
 'reduceOnly': None,
 'fees': [{'currency': 'USDC', 'cost': None}],
 'stopPrice': None}

In [89]:
bingx.fetch_balance()["USDT"]

{'free': 1.5376685963944254, 'used': 14.25, 'total': 15.787668596394425}

In [101]:
c= bingx.fetch_trades("USDC/USDT", limit=2)

In [110]:
c

[{'id': '138725540',
  'info': {'id': '138725540',
   'price': '0.9993',
   'qty': '85.369',
   'time': '1752893280515',
   'buyerMaker': False},
  'timestamp': 1752893280515,
  'datetime': '2025-07-19T02:48:00.515Z',
  'symbol': 'USDC/USDT',
  'order': None,
  'type': None,
  'side': 'buy',
  'takerOrMaker': 'taker',
  'price': 0.9993,
  'amount': 85.369,
  'cost': 85.3092417,
  'fee': {'cost': None, 'currency': None},
  'fees': []},
 {'id': '138725541',
  'info': {'id': '138725541',
   'price': '0.9993',
   'qty': '53.488',
   'time': '1752893280520',
   'buyerMaker': True},
  'timestamp': 1752893280520,
  'datetime': '2025-07-19T02:48:00.520Z',
  'symbol': 'USDC/USDT',
  'order': None,
  'type': None,
  'side': 'sell',
  'takerOrMaker': 'taker',
  'price': 0.9993,
  'amount': 53.488,
  'cost': 53.4505584,
  'fee': {'cost': None, 'currency': None},
  'fees': []}]

In [112]:
c[0]['side']

'buy'

In [111]:
c[0]['price']

0.9993